<a href="https://colab.research.google.com/github/vishalpuri07/VISHAL_INFO5731_FALL2021/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment. (Use 500 records amount all the data you collected).

In [ ]:
# Upload the older csv files that contains the data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving data_lemmatized.csv to data_lemmatized.csv


In [2]:
import pandas as pd
df = pd.read_csv("data_lemmatized.csv") # Contains 500 tweets for search word "Data Mining"

import numpy as np
from numpy import nan
# Since the list new contains nan values now that it is extracted from csv. remove all of those.
new_list = df.values.tolist()
for i in range(0,500):
  del new_list[i][0] # removes the index
  new_list[i] = list(set(new_list[i])) # removes all the nan and bring it to 1
  if new_list[i].count(nan) == 1: # remove all nan values
    new_list[i].remove(nan)
  else:
    1 ==1
    
data_lemmatized = new_list
data_string = ('.'.join(str(x) for x in new_list))

In [53]:
import gensim
import gensim.corpora as corpora
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as the 500 reviews (abstracts, or tweets). 


In [9]:
# 1. Count the frequency of all the N-grams (N=3)
import nltk
nltk.download('punkt')
from textblob import TextBlob
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

tri_grams=TextBlob(data_string).ngrams(3)

hist = dict()
for i in tri_grams:
  i=tuple(i)
  hist[i]=hist.get(i,0)+1

df_trigram_frequency = pd.DataFrame.from_dict(hist, orient = 'index', columns=['tri-grams frequency'])
df_trigram_frequency

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,tri-grams frequency
"('family, 'mining, 'data)",133
"('mining, 'data, 'child)",133
"('data, 'child, 'company)",132
"('child, 'company, 'run)",132
"('company, 'run, 'call)",133
...,...
"('level, 'protection, 'https)",1
"('protection, 'https, 'internet)",1
"('https, 'internet, 'back)",1
"('internet, 'back, 'discuss)",1


In [8]:
# 2. Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

# Generate the bigram and add it as a dictionary that will be later used to create the dataframe.
bi_grams=TextBlob(data_string).ngrams(2)

hist={}
for i in bi_grams:
  i =tuple(i)
  hist[i]=hist.get(i,0)+1

# calculate the probabilities of those bigrams based on the given formula
bigram_probabilities= {}
monty = TextBlob(data_string)
for key, val in hist.items():
  bigram_probabilities[key]=val/monty.words.count(key[0])


df_bigram_probabilities = pd.DataFrame.from_dict(bigram_probabilities, orient = 'index', columns=['Bigrams Probabilities']) # Keys are rows and therefore the orient is selected as "index"
df_bigram_probabilities

,Bigrams Probabilities
"('family, 'mining)",0.847134
"('mining, 'data)",0.530120
"('data, 'child)",0.655172
"('child, 'company)",0.956522
"('company, 'run)",0.796407
...,...
"('field, 'co)",0.200000
"('co, 'isp)",0.005495
"('protection, 'https)",0.200000
"('https, 'internet)",0.012195


3. Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as the 500 reviews (abstracts, or tweets).

In [37]:
# 
import spacy
nlp = spacy.load("en")
from sklearn.feature_extraction.text import CountVectorizer
# Print all the nouns from the each tweets.
noun = [] # Create empty noun list
for i in nlp(data_string).noun_chunks:
  noun.append(i.text)
print(noun) 

# Create a dataframe for the noun that will be transformed later.
df_noun = pd.DataFrame(noun, columns=['noun'])

# Create the word vectorizer and sparse matrix. 
word_vectorizer = CountVectorizer(ngram_range=(3,3))
sparse_matrix = word_vectorizer.fit_transform(df_noun['noun'].values.astype('U')) # Create sparse matrix using scikit learn vectorize.

# Convert to array
total_frequencies = sum(sparse_matrix).toarray()[0]
df_noun_frequencies = pd.DataFrame(total_frequencies, index=word_vectorizer.get_feature_names(),columns=['Frequency']) # Create a dataframe.

# Finally create the frequencies dataframe that has the relative probabilities.
df_noun_frequencies['Noun Probabilities'] = df_noun_frequencies/df_noun_frequencies.max()
df_noun_frequencies.drop(columns = ["Frequency"])

["'family", 'mining', "'company", 'bitcoin', "'use", 'energy', "'https", 'mining', "'data", "'child', 'company", "'run", 'call', "school'].['social_media", "'datum", 'mining', "'ivanka", 'trump', "big', 'source", "'bitcoin", "'overtake", "'month", "'world", "'mining", "'data", "'child', 'company", 'call', 'focus', "'resource", "'care", "new', 'band", "'area", "'role", 'community', "'datum", 'mining', "'people", 'theywritecode', "'librarian", "'obsess", 'mining', "'data", "'child', 'company", "'run", 'call', 'mining', "'data", "'child', 'company", "'run", 'call', 'mining', 'nsagov', 'mining', "'data", "'child', 'company", "'run", 'call', "'mining", "'riser", "'rig", "'delivery", "'sad', 'crypto', 'https', 'lock', 'clip', 'pcie'].['family', 'mining', 'data', 'child', 'company", "'run", 'call', 'mining', "'data", "'child', 'company", "'run", 'call', 'mining', "'data", "'child', 'company", "'run", 'call', 'mining', "'data", "'child', 'company", "'run", 'call', 'mining', "'data", "'child', 

,Noun Probabilities
adcjo bitcoin sy,0.111111
archive iatlzmwm utilise,0.111111
bitcoin launch co,0.111111
bitcoin sy eth,0.111111
bullish qhatwpr company,0.111111
call datum project,0.111111
character invalid https,0.111111
child co company,0.111111
child yuxa terrorist,0.111111
clip pcie family,0.111111


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [69]:
# Import original tweets csv. 
from google.colab import files
uploaded = files.upload()

Saving tweets.csv to tweets (1).csv


In [92]:
import math

idf = 0 # Set inital value of idf_value to zero 
def value(j,k):
  tf=j.count(k)/len(j.split(' '))
  if(tf != 0):
    idf = math.log(len(j.split(' ')))/j.count(k)
    return tf *  idf
  else:
    return 0;
  

strings=pd.read_csv('tweets.csv')['Text'].values.tolist() # Convert each tweets to list

distinct_words=set([i for j in strings for i in j.split(' ')]) # find distinct words using set

tf_times_idf=pd.DataFrame(distinct_words,columns=['terms']) # Create a dataframe

# Create a for loop so that 
element = 0
for i in strings:
  tf_times_idf[str(element)]=tf_times_idf['terms'].apply(lambda x: value(i,x))
  element+= 1
tf_times_idf

,terms,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,,0.144977,0.173287,0.132419,0.197304,0.16666,0.207076,0.144977,0.16666,0.132419,0.136326,0.160576,0.144977,0.149787,0.144977,0.346574,0.140502,0.149787,0.207076,0.207076,0.144977,0.173287,0.132419,0.144977,0.144977,0.207076,0.207076,0.244136,0.132419,0.144977,0.140502,0.144977,0.149787,0.144977,0.15497,0.144977,0.144977,0.132419,0.136326,0.132419,...,0.136326,0.136326,0.149787,0.173287,0.16666,0.140502,0.15497,0.188504,0.140502,0.136326,0.15497,0.149787,0.144977,0.16666,0.136326,0.15497,0.136326,0.188504,0.136326,0.136326,0.244136,0.160576,0.144977,0.140502,0.140502,0.140502,0.140502,0.140502,0.140502,0.140502,0.140502,0.140502,0.140502,0.207076,0.136326,0.132419,0.125311,0.173287,0.188504,0.144977
1,@stoplapdspying:,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.15497,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,#Fiverr:,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"bitcoin,",0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,@Uniswap,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.136326,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.136326,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.136326,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [98]:
# Import necessary modules
import numpy.linalg as alg
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import numpy as np
import re


In [112]:
tweet_list = pd.read_csv('tweets.csv')['Text'].values.tolist() # Convert to the list all the text.

vectorizer = CountVectorizer(stop_words = stopwords.words('english'))
converted_array = vectorizer.fit_transform(tweet_list).toarray() # Using CountVectorizer create an array
transform_array = vectorizer.transform(["Natural language"]).toarray() # Create a transformed array

variable = lambda i,j : np.inner(i,j)/(alg.norm(i)*alg.norm(j))
result = []
for i in converted_array:
  for j in transform_array:
    cosine = variable(i, j)
    result.append(cosine)

df['Cosine Similarity'] = pd.Series(result).values
df["Rank"] = df["Cosine Similarity"].rank()
df

,Cosine Similarity,Rank
0,0.0,250.0
29,0.0,250.0
28,0.0,250.0
1,0.0,250.0
2,0.0,250.0
...,...,...
496,0.0,250.0
497,0.0,250.0
484,0.0,250.0
237,0.0,250.0


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each of the 500 review (if you collected the data from citeseer, you can think of other text classfication tasks instead of sentiment analysis) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 
